# Quick Note:
I somehow lost all the work I did last Sunday. This included the logistic and linear regression. Unfortunatly I don't have the time to go back and redo it but I was able to use my slides that I presented with to add the relevant data into my report


Resources used:

https://www.pro-football-reference.com/

nfl_data_py

https://www.kaggle.com/datasets/tobycrabtree/nfl-scores-and-betting-data/data

https://www.kaggle.com/datasets/sampaynedataanalyst/nfl-regular-season-defensive-stats-2004-2022

https://www.kaggle.com/datasets/sampaynedataanalyst/nfl-regular-season-offensive-stats-2004-2022



In [13]:
'''
!pip install nfl_data_py
!pip install pandas
!pip install matplotlib
!pip install sportsreference
!pip install seaborn
!pip install numpy
!pip install sklearn
!pip install xgboost
!pip install openpyxl
'''

In [18]:
import nfl_data_py as nfl
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('dark_background')
import numpy as np
import seaborn as sns
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import brier_score_loss
from sklearn import linear_model
from xgboost import XGBClassifier
from IPython.display import display, HTML

# Output Settings

In [3]:
# set the max columns to none
pd.set_option('display.max_columns', None)

# Import all the CSV files used

In [4]:
# This cell block takes more time so it's put here to avoid long rung times in case
# I add more files
years_list = [2016, 2017, 2018, 2019, 2020, 2021, 2022]

# Contains each player and their associated team and some other info
season_roster = nfl.import_seasonal_rosters(years_list)

# Get's combine details to get more info on players
combine_details = nfl.import_combine_data() ## GET COMBINE DATA HERE

# Stats for each player
player_stats = nfl.import_seasonal_data(years_list)

# Imports info about team such as logo and different abbrev
team_info = nfl.import_team_desc()

# Get Defensive Player Info

In [5]:
season_roster = season_roster.drop(columns=['depth_chart_position',
                                            'jersey_number',
                                            'status',
                                            'first_name',
                                            'last_name',
                                            'player_id',
                                            'espn_id',
                                            'sportradar_id',
                                            'yahoo_id',
                                            'rotowire_id',
                                            'pff_id',
                                            'pfr_id',
                                            'fantasy_data_id',
                                            'sleeper_id',
                                            'headshot_url',
                                            'ngs_position',
                                            'week',
                                            'game_type',
                                            'status_description_abbr',
                                            'football_name',
                                            'esb_id',
                                            'gsis_it_id',
                                            'smart_id',
                                            'entry_year',
                                            'rookie_year',
                                            'draft_club',
                                            'draft_number'
                                           ])


season_roster = season_roster.drop(season_roster[season_roster['position'] == 'OL'].index)
season_roster = season_roster.drop(season_roster[season_roster['position'] == 'QB'].index)
season_roster = season_roster.drop(season_roster[season_roster['position'] == 'RB'].index)
season_roster = season_roster.drop(season_roster[season_roster['position'] == 'WR'].index)
season_roster = season_roster.drop(season_roster[season_roster['position'] == 'K'].index)
season_roster = season_roster.drop(season_roster[season_roster['position'] == 'LS'].index)
season_roster = season_roster.drop(season_roster[season_roster['position'] == 'P'].index)
season_roster = season_roster.drop(season_roster[season_roster['position'] == 'TE'].index)


In [6]:
# Helpful abbreviations and such for each team.
nfl_teams = pd.read_csv('nfl_teams.csv')

# Same as above with some edits to match another DF
nfl_team_names = pd.read_csv('nfl_team_names.csv')

# NFL offensive stats and records
nfl_offensive = pd.read_excel('NFLRegSeasonOffenseStats.xlsx')

# NFL Defensive stats and records
nfl_defensive = pd.read_excel('NFLRegSeasonDefenseStats.xlsx')

defensive_stats = pd.read_csv('16to22DStats.csv')

# Dataset was taken using nfl_data_py and then saved into a .csv after the
# get defensive player info section was ran (this section can be found
# in the bottom of the project in the no longer used section)
d_roster = pd.read_csv('defensive_roster.csv')

   # Create the master DB for teams

In [7]:
# Merge the two together
master_db = pd.merge(nfl_offensive, nfl_defensive, on=['Team', 'Year'])

# Drop everything except the desired years
master_db = master_db.loc[(master_db['Year'] >= years_list[0])
                         & (master_db['Year'] <= years_list[-1])]

# Drop undsesired rows
master_db = master_db.drop(columns=['iso_code_x',
                                    'iso_code_y',
                                    'Conference_x',
                                    'Conference_y',
                                    'City_x',
                                    'City_y',
                                    'State_x',
                                    'State_y',
                                    'Wins_y',
                                    'Losses_y',
                                    'Ties_y'
                                   ])
master_db = master_db.rename(columns={'Wins_x': 'Wins',
                                      'Losses_x': 'Losses',
                                      'Ties_x': 'Ties'})



# Create the Correlation Matrix

In [8]:
def print_correlation_matrix_original(df):
    # The highest correlation's
    # 1. Passing_TDs 0.568530
    # 2. Interceptions Thrown -0.486708
    # 3. Pass Defelections 0.468254
    # 4. Interceptions 0.437715
    # 5. Rushing TDs 0.427232
    # 6. Sacks 0.405297
    
    temp_df = df.drop(columns="Team")
    correlation = temp_df.corr(method = 'pearson')
    display(correlation)

In [9]:
def print_correlation_matrix_defence(df):
    # The highest correlation's
    # 1. Passing_TDs 0.568530
    # 2. Interceptions Thrown -0.486708
    # 3. Pass Defelections 0.468254
    # 4. Interceptions 0.437715
    # 5. Rushing TDs 0.427232
    # 6. Sacks 0.405297
    
    temp_df = df.drop(columns=['season_x', 'player_name', 'Tm', 'Pos', 'season_y', 'pos', 'school'])
    correlation = temp_df.corr(method = 'pearson')
    display(correlation)

# Clean combine data

In [10]:
df = combine_details
df.to_csv("combinedata.csv")
df = df.drop(columns=[
    'draft_year',
    'draft_team',
    'draft_round',
    'draft_ovr',
    'pfr_id',
    'cfb_id'
])

# Updates height to inches and an int
for index, row in df.iterrows():
    if row['ht']:
        ht_list = row['ht'].split('-')
        ht_inch = (int(ht_list[0]) * 12) + int(ht_list[1])
        df.at[index,'ht'] = ht_inch



de_combine_data = df[df['pos'] == 'DE']
dt_combine_data = df[df['pos'] == 'DT']
olb_combine_data = df[df['pos'] == 'OLB']
ilb_combine_data = df[df['pos'] == 'ILB']
cb_combine_data = df[df['pos'] == 'CB']
s_combine_data = df[df['pos'] == 'S']

de_combine_data['ht'].fillna(int(df['ht'].mean()), inplace=True)
de_combine_data['wt'].fillna(int(df['wt'].mean()), inplace=True)
de_combine_data['forty'].fillna(int(df['forty'].mean()), inplace=True)
de_combine_data['forty'].fillna(int(df['forty'].mean()), inplace=True)
de_combine_data['bench'].fillna(int(df['bench'].mean()), inplace=True)
de_combine_data['broad_jump'].fillna(int(df['broad_jump'].mean()), inplace=True)
de_combine_data['cone'].fillna(int(df['cone'].mean()), inplace=True)
de_combine_data['shuttle'].fillna(int(df['shuttle'].mean()), inplace=True)

dt_combine_data['ht'].fillna(int(df['ht'].mean()), inplace=True)
dt_combine_data['wt'].fillna(int(df['wt'].mean()), inplace=True)
dt_combine_data['forty'].fillna(int(df['forty'].mean()), inplace=True)
dt_combine_data['forty'].fillna(int(df['forty'].mean()), inplace=True)
dt_combine_data['bench'].fillna(int(df['bench'].mean()), inplace=True)
dt_combine_data['broad_jump'].fillna(int(df['broad_jump'].mean()), inplace=True)
dt_combine_data['cone'].fillna(int(df['cone'].mean()), inplace=True)
dt_combine_data['shuttle'].fillna(int(df['shuttle'].mean()), inplace=True)

olb_combine_data['ht'].fillna(int(df['ht'].mean()), inplace=True)
olb_combine_data['wt'].fillna(int(df['wt'].mean()), inplace=True)
olb_combine_data['forty'].fillna(int(df['forty'].mean()), inplace=True)
olb_combine_data['forty'].fillna(int(df['forty'].mean()), inplace=True)
olb_combine_data['bench'].fillna(int(df['bench'].mean()), inplace=True)
olb_combine_data['broad_jump'].fillna(int(df['broad_jump'].mean()), inplace=True)
olb_combine_data['cone'].fillna(int(df['cone'].mean()), inplace=True)
olb_combine_data['shuttle'].fillna(int(df['shuttle'].mean()), inplace=True)

ilb_combine_data['ht'].fillna(int(df['ht'].mean()), inplace=True)
ilb_combine_data['wt'].fillna(int(df['wt'].mean()), inplace=True)
ilb_combine_data['forty'].fillna(int(df['forty'].mean()), inplace=True)
ilb_combine_data['forty'].fillna(int(df['forty'].mean()), inplace=True)
ilb_combine_data['bench'].fillna(int(df['bench'].mean()), inplace=True)
ilb_combine_data['broad_jump'].fillna(int(df['broad_jump'].mean()), inplace=True)
ilb_combine_data['cone'].fillna(int(df['cone'].mean()), inplace=True)
ilb_combine_data['shuttle'].fillna(int(df['shuttle'].mean()), inplace=True)

cb_combine_data['ht'].fillna(int(df['ht'].mean()), inplace=True)
cb_combine_data['wt'].fillna(int(df['wt'].mean()), inplace=True)
cb_combine_data['forty'].fillna(int(df['forty'].mean()), inplace=True)
cb_combine_data['forty'].fillna(int(df['forty'].mean()), inplace=True)
cb_combine_data['bench'].fillna(int(df['bench'].mean()), inplace=True)
cb_combine_data['broad_jump'].fillna(int(df['broad_jump'].mean()), inplace=True)
cb_combine_data['cone'].fillna(int(df['cone'].mean()), inplace=True)
cb_combine_data['shuttle'].fillna(int(df['shuttle'].mean()), inplace=True)

s_combine_data['ht'].fillna(int(df['ht'].mean()), inplace=True)
s_combine_data['wt'].fillna(int(df['wt'].mean()), inplace=True)
s_combine_data['forty'].fillna(int(df['forty'].mean()), inplace=True)
s_combine_data['forty'].fillna(int(df['forty'].mean()), inplace=True)
s_combine_data['bench'].fillna(int(df['bench'].mean()), inplace=True)
s_combine_data['broad_jump'].fillna(int(df['broad_jump'].mean()), inplace=True)
s_combine_data['cone'].fillna(int(df['cone'].mean()), inplace=True)
s_combine_data['shuttle'].fillna(int(df['shuttle'].mean()), inplace=True)

# Combine them all back together
combine_data = pd.concat([s_combine_data, cb_combine_data, 
                           ilb_combine_data, olb_combine_data, 
                           dt_combine_data, de_combine_data])


C:\Users\seaha\AppData\Local\Temp\ipykernel_22640\3919438436.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  de_combine_data['ht'].fillna(int(df['ht'].mean()), inplace=True)
C:\Users\seaha\AppData\Local\Temp\ipykernel_22640\3919438436.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  de_combine_data['wt'].fillna(int(df['wt'].mean()), inplace=True)
C:\Users\seaha\AppData\Local\Temp\ipykernel_22640\3919438436.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#

# Getting team averages at every position

In [11]:
# Merge the defenders stats with the combine stats
# need to keep this one as is for matrix
defensive_roster = pd.merge(defensive_stats, combine_data, on=['player_name'])

# d_roster was saved from defensive roster above and some changes were made manually
de_roster = d_roster[d_roster['pos'] == 'DE']
dt_roster = d_roster[d_roster['pos'] == 'DT']
ilb_roster = d_roster[d_roster['pos'] == 'ILB']
olb_roster = d_roster[d_roster['pos'] == 'OLB']
cb_roster = d_roster[d_roster['pos'] == 'CB']
s_roster = d_roster[d_roster['pos'] == 'S']

# What averages we need for each team at each position
# DE's = HT, WT, Vertical, forty
# DT's WT, broad, vertical, forty
# ILB's - vertical, broad, weight
# OLBs = weight, broad, ht, vertical
# CB's - vertical, broad
# S's = vertical, cone, wt

# Get the season averages for each team at each position group
de = de_roster.groupby(['season', 'Tm'], as_index=False)[['forty', 'ht', 'wt', 'vertical']].mean()
dt = dt_roster.groupby(['season', 'Tm'], as_index=False)[['wt', 'broad_jump', 'vertical', 'forty']].mean()
ilb = ilb_roster.groupby(['season', 'Tm'], as_index=False)[['vertical', 'broad_jump', 'wt']].mean()
olb = olb_roster.groupby(['season', 'Tm'], as_index=False)[['wt', 'broad_jump', 'ht', 'vertical']].mean()
cb = cb_roster.groupby(['season', 'Tm'], as_index=False)[['vertical', 'broad_jump']].mean()
s = s_roster.groupby(['season', 'Tm'], as_index=False)[['vertical', 'wt', 'cone']].mean()

# Fill na with average of column
de = de.fillna(de.mean(numeric_only=True))
dt = dt.fillna(dt.mean(numeric_only=True))
ilb = ilb.fillna(ilb.mean(numeric_only=True))
olb = olb.fillna(olb.mean(numeric_only=True))
cb = cb.fillna(cb.mean(numeric_only=True))
s = s.fillna(s.mean(numeric_only=True))

# Rename all the columns before putting into one db
de.rename(columns={'forty': 'DEfortyAvg', 'ht': 'DEhtAvg', 
                   'wt': 'DEwtAvg', 'vertical': 'DEverticalAvg'}, inplace=True)

dt.rename(columns={'forty': 'DTfortyAvg', 'broad_jump': 'DTbroadAvg', 
                   'wt': 'DTwtAvg', 'vertical': 'DTverticalAvg'}, inplace=True)

ilb.rename(columns={'broad_jump': 'ILBbroadAvg','wt': 'ILBwtAvg',
                    'vertical': 'ILBverticalAvg'}, inplace=True)

olb.rename(columns={'ht': 'OLBhtAvg', 'broad_jump': 'OLBbroadAvg', 
                   'wt': 'OLBwtAvg', 'vertical': 'OLBverticalAvg'}, inplace=True)

cb.rename(columns={'broad_jump': 'CBbroadAvg', 'vertical': 'CBverticalAvg'}, inplace=True)

s.rename(columns={'cone': 'SconeAvg', 'wt': 'SwtAvg',
                  'vertical': 'SverticalAvg'}, inplace=True)

# Merge into a single DB doing three different DB's
merged_all = pd.merge(de, dt, on=['season', 'Tm'])
merged_all = pd.merge(merged_all, olb, on=['season', 'Tm'])
merged_all = pd.merge(merged_all, cb, on=['season', 'Tm'])
merged_all = pd.merge(merged_all, s, on=['season', 'Tm'])
merged_noilb = merged_all
merged_all = pd.merge(merged_all, ilb, on=['season', 'Tm'])
merged_all.rename(columns={'Tm': 'team_name'}, inplace=True)
merged_noilb.rename(columns={'Tm': 'team_name'}, inplace=True)


# Setting up the team wins for each year. Have to mess around with some of the data so
# that each DB has the same naming conventions
master_db_basic = master_db[['Team', 'Year', 'Wins']].copy()
master_db_basic.rename(columns={'Team': 'Tm', 'Year': 'season'}, inplace=True)
master_db_basic = pd.merge(master_db_basic, nfl_team_names, on=['Tm'])
merged_all = pd.merge(master_db_basic, merged_all, on=['season', 'team_name'])
merged_noilb = pd.merge(master_db_basic, merged_noilb, on=['season', 'team_name'])


# Logistic Regression and Linear Regression

In [ ]:
'''
This was around where my linear and logistic regression was. I'm not sure what happened
but all the work I did on Sunday was lost. Sunday is when I worked on the logistic and linear
regression. Unfortunatly I don't have time to redo this part. However I created my presentation
on sunday night when I still had access to all the data and outputs it was providing so I do
still have access to the data I reported on.
'''

# Print out matrix at positions

In [12]:
def position_matrix(defensive_roster):
    de_roster = defensive_roster[defensive_roster['pos'] == 'DE']
    dt_roster = defensive_roster[defensive_roster['pos'] == 'DT']
    ilb_roster = defensive_roster[defensive_roster['pos'] == 'ILB']
    olb_roster = defensive_roster[defensive_roster['pos'] == 'OLB']
    cb_roster = defensive_roster[defensive_roster['pos'] == 'CB']
    s_roster = defensive_roster[defensive_roster['pos'] == 'S']
    
    print("DE's")
    print_correlation_matrix_defence(de_roster)
    # Ints leaders - no correlation worth highlighting
    # PD's = HT = 0.214029, WT = 0.179563
    # Sacks = forty = -0.16631, vertical = 0.164293, and broad jump = 0.144067
    # QB hits = forty = -0.130258, vertical = 0.116204, broad_jump = 0.114900

    print("DT's")
    print_correlation_matrix_defence(dt_roster)
    # Ints leaders - None
    # PD's = None
    # Sacks = wt -0.212267, 40 = -0.125331,bench = 0.116742, vertical = 0.120845, broad = 0.194133, cone = -0.152739
    # QB hits = wt = -0.241188, forty = -0.120472, bench = 0.118849, vertical = 0.143933, broad = 0.219948, 0.163284

    print("ILB's")
    print_correlation_matrix_defence(ilb_roster)
    # Ints leaders - None
    # PD's = vertical = 0.188513, broad = 0.169417
    # Sacks = weight = 0.137233, vertical = 0.146358, 
    # QB hits = weight = 0.171089, vertical = 0.200448, broad = 0.171032

    print("OLB's")
    print_correlation_matrix_defence(olb_roster)
    # Ints leaders - weight -0.214119, bench -109554
    # PD's = weight -0.265915, broad jump = 0.22174
    # Sacks = weight =  0.29339, ht = 0.224814, vertical = 0.251377, broad = 0.238754
    # QB hits = weight = 0.317846, height = 0.225654, vertical = 0.252409, broad_jump = 0.21810

    print("CB's")
    print_correlation_matrix_defence(cb_roster)
    # Ints leaders - none
    # PD's = vertical = 0.128207
    # Sacks = vertical = -0.122768, broad_jump = -0.101390
    # QB hits = none


    print("S's")
    print_correlation_matrix_defence(s_roster)
    # Ints leaders = vertical = 0.106236, cone = -0.174591
    # PD's = none
    # Sacks = wt = 0.113393
    # QB hits = wt = 0.134692

# Print final DF matrix

In [13]:
def final_df_matrix(df):
    temp_df = df.drop(columns=['Tm', 'season', 'team_name_short', 'team_name'])
    correlation = temp_df.corr(method = 'pearson')
    display(correlation)
    

# Put into CSV file

In [14]:
def into_csv(df, file_name):
    df.to_csv(file_name)

# Most everything runs through here

In [24]:
display(merged_all)
# Find the correlation
print_correlation_matrix_original(master_db)
position_matrix(defensive_roster)
final_df_matrix(merged_all)
final_df_matrix(merged_noilb)
display(defensive_roster)
display(combine_data)
#display(merged_all)

,Tm,season,Wins,team_name_short,team_name,DEfortyAvg,DEhtAvg,DEwtAvg,DEverticalAvg,DTwtAvg,DTbroadAvg,DTverticalAvg,DTfortyAvg,OLBwtAvg,OLBbroadAvg,OLBhtAvg,OLBverticalAvg,CBverticalAvg,CBbroadAvg,SverticalAvg,SwtAvg,SconeAvg,ILBverticalAvg,ILBbroadAvg,ILBwtAvg
0,Buffalo Bills,2016,7,Bills,BUF,4.700000,75.000000,269.000000,33.000000,306.200000,108.000000,28.000000,5.068000,249.666667,116.000000,73.666667,34.000000,36.916667,121.857143,37.500000,206.000000,6.720000,31.000000,113.500000,250.000000
1,Buffalo Bills,2017,6,Bills,BUF,4.700000,75.000000,269.000000,33.000000,306.666667,106.833333,27.800000,5.098333,239.000000,122.000000,73.000000,34.750000,32.800000,117.000000,40.500000,213.000000,7.000000,33.250000,118.500000,239.500000
2,Buffalo Bills,2020,13,Bills,BUF,4.875000,76.000000,257.500000,31.000000,307.000000,107.000000,30.166667,5.163333,237.000000,122.250000,72.750000,34.000000,33.714286,120.428571,35.666667,197.666667,7.000000,33.616493,113.500000,243.500000
3,Miami Dolphins,2016,10,Dolphins,MIA,4.710000,77.333333,275.666667,32.833333,313.750000,104.250000,30.500000,5.027500,240.600000,123.200000,74.400000,39.166667,34.125000,125.500000,39.500000,208.666667,7.143333,33.616493,114.000000,242.000000
4,Miami Dolphins,2017,6,Dolphins,MIA,4.745000,75.500000,256.000000,32.250000,310.000000,104.500000,30.125000,5.085000,235.666667,122.333333,73.333333,40.250000,32.700000,122.600000,39.750000,210.250000,7.080000,37.000000,106.333333,243.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,Seattle Seahawks,2017,9,Seahawks,SEA,4.842500,75.250000,267.250000,34.000000,302.200000,108.200000,28.900000,5.098000,249.000000,119.666667,76.000000,32.333333,38.000000,123.500000,32.250000,217.500000,7.235000,28.000000,109.000000,235.000000
132,Seattle Seahawks,2018,10,Seahawks,SEA,4.860000,76.000000,273.000000,34.166667,302.750000,105.500000,28.750000,5.130000,240.500000,121.750000,75.500000,33.833333,38.000000,123.000000,32.666667,202.000000,6.993333,39.500000,127.000000,239.000000
133,Seattle Seahawks,2019,11,Seahawks,SEA,4.773333,76.666667,271.333333,33.833333,302.333333,109.000000,32.333333,5.103333,236.500000,118.500000,74.000000,32.000000,37.000000,122.500000,33.625000,199.500000,7.080000,36.000000,118.000000,242.000000
134,Seattle Seahawks,2020,12,Seahawks,SEA,4.822500,76.500000,271.750000,32.875000,307.000000,104.000000,31.000000,5.210000,239.333333,120.000000,74.333333,32.750000,36.833333,119.666667,34.300000,200.800000,7.000000,32.500000,109.000000,245.000000


,Year,Wins,Losses,Ties,Passing_Attempts,Passing_Completions,Passing_Yards,Passing_TDs,Interceptions_Thrown,Rushing _Attempts,Rushing_Yards,Rushing_TDs,Rushing_Fumbles,Receptions,Receiving_Yards,Receiving_TDs,Reciever_Fumbles,Two_Point_Conversions,Kick_Return_TDs,Punt_Return_TDs,Field_Goals_Attempts,Field_Goals_Made,Extra_Points_Attempts,Extra_Points_Made,Solo_Tackles,Assisted_Tackles,Sacks,Tackles_For_Loss,Pass_Deflections,Interceptions,Interception_TDs,Forced_Fumbles,Fumbles_Recovered,Fumble__TDs,Kicks_Blocked
Year,1.000000,0.063437,0.046505,0.016422,0.079541,0.146970,0.042290,0.033997,0.002472,0.285257,0.293070,0.215694,0.174562,0.169088,0.025648,0.033997,0.337679,0.123899,0.015158,-0.171569,0.066269,0.070640,0.076013,0.074345,0.218240,0.712957,0.125096,-0.323832,0.004090,0.002766,-0.007984,-0.037299,-0.004525,-0.088327,-0.054098
Wins,0.063437,1.000000,-0.986102,-0.171655,0.013625,0.251698,0.356603,0.568530,-0.486708,0.364956,0.319138,0.427232,-0.020349,0.234545,0.334868,0.568530,0.012535,0.095514,0.043593,-0.001745,0.207449,0.239120,0.730668,0.726507,-0.289923,-0.083842,0.405297,0.065144,0.468254,0.437715,0.193145,0.226555,0.335485,0.148058,0.041928
Losses,0.046505,-0.986102,1.000000,0.089975,-0.010892,-0.240600,-0.357596,-0.575465,0.495771,-0.310751,-0.277130,-0.404387,0.061448,-0.223513,-0.341767,-0.575465,0.059668,-0.089310,-0.038772,-0.020681,-0.191285,-0.219330,-0.727142,-0.721090,0.342500,0.181594,-0.398511,-0.129698,-0.456204,-0.421026,-0.178122,-0.226572,-0.347192,-0.156046,-0.049047
Ties,0.016422,-0.171655,0.089975,1.000000,0.159354,0.082501,0.029656,-0.058298,0.021077,-0.102636,-0.069829,-0.056235,0.046171,0.113396,0.060695,-0.058298,-0.004033,0.083280,-0.069038,-0.027236,0.046188,0.008904,-0.136720,-0.149076,0.120116,0.078985,0.083577,0.106753,-0.040635,-0.155514,-0.122988,0.074883,0.060686,-0.065419,0.047466
Passing_Attempts,0.079541,0.013625,-0.010892,0.159354,1.000000,0.883870,0.702589,0.299477,0.191778,-0.509263,-0.540024,-0.225755,-0.161492,0.864299,0.714960,0.299477,0.162678,0.198888,-0.023426,-0.063011,0.211548,0.160352,0.026851,0.015829,0.043074,-0.047776,0.075513,-0.021094,0.014975,-0.008877,-0.001940,-0.047523,0.000627,-0.102140,-0.025266
Passing_Completions,0.146970,0.251698,-0.240600,0.082501,0.883870,1.000000,0.828115,0.476580,-0.063057,-0.401509,-0.422932,-0.067344,-0.178554,0.961071,0.820205,0.476580,0.186734,0.172788,-0.022576,-0.038397,0.231533,0.199712,0.263186,0.256645,-0.061208,-0.015682,0.091909,-0.123584,0.094824,0.070294,0.060249,-0.055361,0.061269,-0.115756,-0.070622
Passing_Yards,0.042290,0.356603,-0.357596,0.029656,0.702589,0.828115,1.000000,0.686873,-0.087484,-0.328160,-0.298502,0.067030,-0.160726,0.798132,0.984712,0.686873,0.104694,0.137835,-0.043825,-0.037643,0.237980,0.184155,0.517346,0.498072,-0.084358,-0.083635,0.104182,-0.088799,0.091939,0.042904,0.045720,0.028325,0.073017,-0.033138,-0.032261
Passing_TDs,0.033997,0.568530,-0.575465,-0.058298,0.299477,0.476580,0.686873,1.000000,-0.273609,-0.069480,0.036589,0.123638,-0.064236,0.464462,0.686845,1.000000,0.045643,0.190499,0.019095,-0.069675,-0.100081,-0.076944,0.802299,0.777219,-0.081269,-0.127099,0.163976,0.025073,0.168889,0.191426,0.096003,0.114542,0.210785,0.029670,0.001401
Interceptions_Thrown,0.002472,-0.486708,0.495771,0.021077,0.191778,-0.063057,-0.087484,-0.273609,1.000000,-0.224642,-0.270394,-0.271239,0.015839,-0.062081,-0.079724,-0.273609,0.073374,0.007157,-0.063152,-0.031978,-0.121307,-0.153027,-0.400531,-0.407545,0.229361,0.032184,-0.014894,0.250147,-0.224672,-0.182593,-0.082066,0.048244,-0.084429,0.066984,-0.089767
Rushing _Attempts,0.285257,0.364956,-0.310751,-0.102636,-0.509263,-0.401509,-0.328160,-0.069480,-0.224642,1.000000,0.863515,0.568970,0.377102,-0.396926,-0.359336,-0.069480,0.052407,0.069177,-0.002279,-0.120806,0.156748,0.157914,0.260929,0.264906,-0.137773,0.219237,0.117155,-0.095554,0.179677,0.187800,0.055163,0.177462,0.184263,0.195560,0.045910


DE's


,Age,Int,PD,Sk,QBHits,ht,wt,forty,bench,vertical,broad_jump,cone,shuttle
Age,1.000000,-0.018907,0.188646,0.111782,0.127642,0.040329,0.212797,0.116556,0.030653,-0.060844,-0.178536,-0.068159,0.011339
Int,-0.018907,1.000000,0.292965,0.150086,0.123416,0.098115,-0.008505,-0.024091,-0.006620,0.058550,0.090795,-0.055665,0.025581
PD,0.188646,0.292965,1.000000,0.374249,0.373003,0.214029,0.179563,-0.069585,-0.037943,0.017079,0.009929,-0.021254,0.080976
Sk,0.111782,0.150086,0.374249,1.000000,0.887816,0.046820,-0.007536,-0.166631,0.108989,0.164293,0.144067,-0.025434,-0.006837
QBHits,0.127642,0.123416,0.373003,0.887816,1.000000,0.026552,0.014750,-0.130258,0.106757,0.116204,0.114900,0.030313,0.029442
ht,0.040329,0.098115,0.214029,0.046820,0.026552,1.000000,0.347941,0.125281,-0.092218,-0.157334,0.119237,0.007168,0.127577
wt,0.212797,-0.008505,0.179563,-0.007536,0.014750,0.347941,1.000000,0.370024,0.154709,-0.330711,-0.203475,0.245958,0.160557
forty,0.116556,-0.024091,-0.069585,-0.166631,-0.130258,0.125281,0.370024,1.000000,0.000961,-0.483339,-0.240069,0.299178,0.230961
bench,0.030653,-0.006620,-0.037943,0.108989,0.106757,-0.092218,0.154709,0.000961,1.000000,0.300664,0.177797,0.138679,0.045998
vertical,-0.060844,0.058550,0.017079,0.164293,0.116204,-0.157334,-0.330711,-0.483339,0.300664,1.000000,0.602192,-0.270546,-0.264546


DT's


...

,Age,Int,PD,Sk,QBHits,ht,wt,forty,bench,vertical,broad_jump,cone,shuttle
Age,1.000000,0.072211,0.027949,-0.006974,0.036294,-0.036943,0.073284,0.211730,0.007715,-0.042490,-0.150402,-0.028029,0.064036
Int,0.072211,1.000000,0.735117,0.140940,0.164862,0.037474,-0.014697,-0.043358,-0.088117,0.106236,-0.009774,-0.174591,-0.075882
PD,0.027949,0.735117,1.000000,0.254724,0.291939,0.023189,-0.012780,-0.016582,-0.091086,0.056316,-0.007102,-0.156255,-0.088117
Sk,-0.006974,0.140940,0.254724,1.000000,0.806534,0.022924,0.113393,-0.031380,0.015254,-0.075487,-0.005508,-0.004644,-0.046872
QBHits,0.036294,0.164862,0.291939,0.806534,1.000000,0.032623,0.134692,0.031268,-0.003646,-0.090862,0.004456,-0.048114,-0.043484
ht,-0.036943,0.037474,0.023189,0.022924,0.032623,1.000000,0.353804,-0.037116,0.015121,0.093964,0.260431,0.000623,-0.021268
wt,0.073284,-0.014697,-0.012780,0.113393,0.134692,0.353804,1.000000,0.088205,0.291612,-0.012444,0.124455,0.165375,0.021232
forty,0.211730,-0.043358,-0.016582,-0.031380,0.031268,-0.037116,0.088205,1.000000,-0.035915,-0.056854,0.168379,-0.053983,0.216404
bench,0.007715,-0.088117,-0.091086,0.015254,-0.003646,0.015121,0.291612,-0.035915,1.000000,0.134551,0.129656,0.134738,-0.026362
vertical,-0.042490,0.106236,0.056316,-0.075487,-0.090862,0.093964,-0.012444,-0.056854,0.134551,1.000000,0.716871,-0.033499,0.004635


,Wins,DEfortyAvg,DEhtAvg,DEwtAvg,DEverticalAvg,DTwtAvg,DTbroadAvg,DTverticalAvg,DTfortyAvg,OLBwtAvg,OLBbroadAvg,OLBhtAvg,OLBverticalAvg,CBverticalAvg,CBbroadAvg,SverticalAvg,SwtAvg,SconeAvg,ILBverticalAvg,ILBbroadAvg,ILBwtAvg
Wins,1.000000,-0.046176,0.057094,0.031243,0.056370,0.010372,-0.099751,0.080157,0.083827,-0.008984,-0.009352,0.012921,-0.103752,0.069795,-0.013675,-0.028602,-0.117133,-0.119247,0.166549,0.166924,0.163583
DEfortyAvg,-0.046176,1.000000,0.111359,0.181273,-0.443916,0.257842,-0.049129,0.013842,0.025849,0.027388,-0.113896,-0.081160,-0.052962,-0.033620,0.138314,-0.235603,-0.020310,0.211869,-0.060914,-0.013574,0.081756
DEhtAvg,0.057094,0.111359,1.000000,0.528103,-0.241821,0.040837,0.210764,0.239672,-0.068640,-0.158057,0.025286,-0.048661,0.070905,0.010689,0.027956,-0.105485,-0.054333,0.140766,0.117686,-0.010867,0.014522
DEwtAvg,0.031243,0.181273,0.528103,1.000000,-0.327594,-0.050494,0.134650,0.162070,-0.133548,-0.001598,0.057644,0.026787,0.031026,-0.028931,-0.138245,-0.000051,0.074327,0.020732,-0.039992,-0.000417,-0.049455
DEverticalAvg,0.056370,-0.443916,-0.241821,-0.327594,1.000000,-0.070777,-0.194343,-0.128020,0.018708,0.027681,0.089773,0.151979,-0.015744,0.127871,-0.129738,-0.100853,-0.027937,-0.162253,-0.098636,-0.056102,-0.012102
DTwtAvg,0.010372,0.257842,0.040837,-0.050494,-0.070777,1.000000,-0.236535,-0.065262,0.377261,-0.114841,0.021006,0.013432,0.007647,0.016379,0.167675,-0.089491,0.007662,0.169121,-0.030536,0.103042,0.115251
DTbroadAvg,-0.099751,-0.049129,0.210764,0.134650,-0.194343,-0.236535,1.000000,0.499329,-0.403379,-0.089443,0.006046,0.017289,0.087561,0.171503,0.035610,0.048515,0.037332,0.261121,0.037141,-0.003429,0.176256
DTverticalAvg,0.080157,0.013842,0.239672,0.162070,-0.128020,-0.065262,0.499329,1.000000,-0.215616,-0.183183,0.101169,-0.039266,0.181358,0.183780,0.111854,-0.023581,-0.146335,0.134594,0.084889,0.036935,0.158171
DTfortyAvg,0.083827,0.025849,-0.068640,-0.133548,0.018708,0.377261,-0.403379,-0.215616,1.000000,0.047543,-0.086905,-0.052652,-0.175765,-0.062026,0.000819,0.014707,0.054377,0.159643,-0.286088,-0.150774,-0.040332
OLBwtAvg,-0.008984,0.027388,-0.158057,-0.001598,0.027681,-0.114841,-0.089443,-0.183183,0.047543,1.000000,-0.180933,0.398019,0.018924,0.023566,-0.085465,0.006347,-0.017945,-0.227668,-0.132739,-0.056305,0.087277


,Wins,DEfortyAvg,DEhtAvg,DEwtAvg,DEverticalAvg,DTwtAvg,DTbroadAvg,DTverticalAvg,DTfortyAvg,OLBwtAvg,OLBbroadAvg,OLBhtAvg,OLBverticalAvg,CBverticalAvg,CBbroadAvg,SverticalAvg,SwtAvg,SconeAvg
Wins,1.000000,-0.022034,0.018483,0.065533,0.026776,0.071637,-0.101047,0.012990,0.057017,-0.031468,0.117154,0.022283,0.025086,0.026752,0.001362,-0.004638,-0.036909,-0.092284
DEfortyAvg,-0.022034,1.000000,0.114433,0.299606,-0.434251,0.156008,-0.033351,-0.020452,-0.050768,0.040947,-0.040353,-0.047668,-0.019900,-0.072240,0.092310,-0.201075,0.015294,0.227186
DEhtAvg,0.018483,0.114433,1.000000,0.357231,-0.196527,0.028796,0.187668,0.140728,-0.099926,-0.117629,0.043252,-0.024326,0.071674,0.022701,0.013287,-0.059757,-0.034599,0.105847
DEwtAvg,0.065533,0.299606,0.357231,1.000000,-0.328009,-0.028115,0.156135,0.066673,-0.232790,-0.040720,0.092922,0.080882,0.077857,-0.026944,-0.063317,0.039890,0.140440,0.042789
DEverticalAvg,0.026776,-0.434251,-0.196527,-0.328009,1.000000,-0.043622,-0.097500,-0.089426,0.031389,-0.023936,0.013627,0.083553,-0.034453,0.110956,-0.079381,-0.070810,-0.010046,-0.103593
DTwtAvg,0.071637,0.156008,0.028796,-0.028115,-0.043622,1.000000,-0.173074,-0.149060,0.387251,-0.085208,0.031808,-0.053625,0.031425,-0.019413,0.152262,-0.075235,-0.062687,0.115325
DTbroadAvg,-0.101047,-0.033351,0.187668,0.156135,-0.097500,-0.173074,1.000000,0.424585,-0.421559,-0.079421,0.000305,0.002991,0.086966,0.177466,0.078240,0.092977,0.052910,0.241184
DTverticalAvg,0.012990,-0.020452,0.140728,0.066673,-0.089426,-0.149060,0.424585,1.000000,-0.131670,-0.157941,0.050212,-0.023584,0.107842,0.125292,0.075270,0.000737,-0.087305,0.080471
DTfortyAvg,0.057017,-0.050768,-0.099926,-0.232790,0.031389,0.387251,-0.421559,-0.131670,1.000000,-0.035377,-0.109155,-0.120084,-0.176189,-0.061738,-0.040137,-0.074690,-0.044406,0.080552
OLBwtAvg,-0.031468,0.040947,-0.117629,-0.040720,-0.023936,-0.085208,-0.079421,-0.157941,-0.035377,1.000000,-0.077476,0.367310,0.005598,0.013925,-0.040517,-0.007567,-0.054812,-0.217648


,season_x,player_name,Tm,Age,Pos,Int,PD,Sk,QBHits,season_y,pos,school,ht,wt,forty,bench,vertical,broad_jump,cone,shuttle
0,2016,Kwon Alexander,TAM,22,MLB,1,7,3.0,7,2015,OLB,LSU,73,227.0,4.55,24.0,36.0,121.0,7.14,4.20
1,2017,Kwon Alexander,TAM,23,MLB,3,4,0.0,3,2015,OLB,LSU,73,227.0,4.55,24.0,36.0,121.0,7.14,4.20
2,2018,Kwon Alexander,TAM,24,MLB,0,2,1.0,2,2015,OLB,LSU,73,227.0,4.55,24.0,36.0,121.0,7.14,4.20
3,2019,Kwon Alexander,SFO,25,LLB,1,4,0.5,1,2015,OLB,LSU,73,227.0,4.55,24.0,36.0,121.0,7.14,4.20
4,2020,Kwon Alexander,2TM,26,LLB,0,4,1.0,2,2015,OLB,LSU,73,227.0,4.55,24.0,36.0,121.0,7.14,4.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3979,2022,Matthew Butler,LVR,23,DT,0,0,0.5,1,2022,DT,Tennessee,76,297.0,5.00,17.0,32.0,112.0,7.89,4.81
3980,2022,Christopher Hinton,LAC,22,DT,0,0,0.0,1,2022,DT,Michigan,76,305.0,5.28,20.0,31.5,106.0,7.00,4.00
3981,2022,Eric Johnson,IND,24,DT,0,0,0.0,0,2000,S,Yale,72,215.0,4.60,12.0,38.0,121.0,6.96,4.17
3982,2022,Chase Lucas,DET,25,CB,0,0,0.0,0,2022,CB,Arizona St.,71,180.0,4.48,20.0,39.0,128.0,7.00,4.00


,season,player_name,pos,school,ht,wt,forty,bench,vertical,broad_jump,cone,shuttle
12,2000,Rashidi Barnes,S,Colorado,72,208.0,4.62,10.0,35.0,114.0,6.92,4.32
20,2000,Rogers Beckett,S,Marshall,73,207.0,4.62,15.0,39.5,119.0,6.48,4.29
23,2000,Gary Berry,S,Ohio State,71,204.0,4.55,20.0,NaN,114.0,7.00,4.00
25,2000,Matt Bowen,S,Iowa,73,201.0,4.49,12.0,33.0,115.0,7.06,4.09
32,2000,Mike Brown,S,Nebraska,70,204.0,4.59,14.0,36.0,116.0,6.93,4.01
...,...,...,...,...,...,...,...,...,...,...,...,...
7831,2023,Viliami Fehoko Jr.,DE,San Jose St.,76,276.0,4.00,24.0,NaN,114.0,7.00,4.00
7848,2023,Tyler Lacy,DE,Oklahoma St.,76,279.0,5.11,30.0,28.5,116.0,7.60,4.00
7866,2023,Isaiah McGuire,DE,Missouri,76,268.0,4.76,20.0,36.5,122.0,7.39,4.00
7879,2023,Mike Morris,DE,Michigan,77,275.0,4.95,22.0,28.5,110.0,7.00,4.00


In [42]:
# Puts stuff into csv's for easier viewing.
#seasonal_roster.to_csv('seasonal_roster.csv')

#win_totals.to_csv('win_totals.csv')

#player_stats.to_csv('player_stats.csv')

#wins.to_csv('wins.csv')

#master_db.to_csv('test2.csv')


# Code blocks no longer used. Found different solutions or saved the output into a .csv and uploaded the csv directly.

In [ ]:
## I have since found datasets that contain the wins/losses and this is no longer needed.

def get_team_wins(from_date, to_date):
    from_date = from_date
    to_date = to_date
    # Data starting in 1966 of every matchup of every week. Used to get team wins.
    team_season_wins = pd.read_csv('spreadspoke_scores.csv')
    
    # Select our date ranges here
    team_season_wins = team_season_wins.loc[(team_season_wins['schedule_season'] >= from_date) 
                                            & (team_season_wins['schedule_season'] <= to_date)]
    # Select only regular season games
    team_season_wins = team_season_wins.loc[team_season_wins['schedule_playoff'] != True]
    
    # Keep some data for weather in case I need it later
    team_season_wins_with_weather = team_season_wins.drop(columns = 
                                             ['schedule_date',
                                              'stadium_neutral',
                                              'stadium',
                                              'over_under_line',
                                              'spread_favorite',
                                              'team_favorite_id'])

    # Drop irrelivent data
    team_season_wins = team_season_wins.drop(columns = 
                                             ['schedule_date',
                                              'stadium_neutral',
                                              'schedule_playoff',
                                              'stadium',
                                              'over_under_line',
                                              'spread_favorite',
                                              'team_favorite_id',
                                              'weather_temperature',
                                              'weather_wind_mph',
                                              'weather_humidity',
                                              'weather_detail'])
    # Here we create our new dataframe that will have rows
    # of each team and their record. Will be used later to
    # continue to add averages per team.
    new_df = {'Season': [],
              'Team': [],
              'Wins': [],
              'Losses': [],
              'Ties': []}
    
    current_season = ""
    
    for index, row in team_season_wins.iterrows():
        home_team = row['team_home']
        home_score = row['score_home']

        away_team = row['team_away']
        away_score = row['score_away']

        if row['schedule_season'] == current_season:
            # This is used to find out where the start of this season is within the index
            season_index = new_df['Season'].index(current_season)
            
            # This gets our team index within that season
            if (home_team in new_df['Team'][season_index:]) & (away_team in new_df['Team'][season_index:]):
                home_team_index = new_df['Team'].index(home_team, season_index)
                away_team_index = new_df['Team'].index(away_team, season_index)
                
                # Add win's losses and ties
                if home_score > away_score:
                    new_df['Wins'][home_team_index] += 1
                    new_df['Losses'][away_team_index] += 1
                elif home_score < away_score:
                    new_df['Wins'][away_team_index] += 1
                    new_df['Losses'][home_team_index] += 1
                else:
                    # It was a tie
                    new_df['Ties'][away_team_index] += 1
                    new_df['Ties'][home_team_index] += 1
            else:
                # The team is not in the dictionary for this season
                # Put them in with 0's for wins and losses
                
                new_df['Season'].append(current_season)
                new_df['Team'].append(home_team)
                new_df['Wins'].append(0)
                new_df['Losses'].append(0)
                new_df['Ties'].append(0)
                home_team_index = len(new_df['Ties']) - 1
                
                new_df['Season'].append(current_season)
                new_df['Team'].append(away_team)
                new_df['Wins'].append(0)
                new_df['Losses'].append(0)
                new_df['Ties'].append(0)
                away_team_index = len(new_df['Ties']) - 1
                
                if home_score > away_score:
                    new_df['Wins'][home_team_index] += 1
                    new_df['Losses'][away_team_index] += 1
                elif home_score < away_score:
                    new_df['Wins'][away_team_index] += 1
                    new_df['Losses'][home_team_index] += 1
                else:
                    # It was a tie
                    new_df['Ties'][away_team_index] += 1
                    new_df['Ties'][home_team_index] += 1

        else:
            # We are at the first game in a new season
            current_season = row['schedule_season']
                
            new_df['Season'].append(current_season)
            new_df['Team'].append(home_team)
            new_df['Wins'].append(0)
            new_df['Losses'].append(0)
            new_df['Ties'].append(0)
            home_team_index = len(new_df['Ties']) - 1

            new_df['Season'].append(current_season)
            new_df['Team'].append(away_team)
            new_df['Wins'].append(0)
            new_df['Losses'].append(0)
            new_df['Ties'].append(0)
            away_team_index = len(new_df['Ties']) - 1

            if home_score > away_score:
                new_df['Wins'][home_team_index] += 1
                new_df['Losses'][away_team_index] += 1
            elif home_score < away_score:
                new_df['Wins'][away_team_index] += 1
                new_df['Losses'][home_team_index] += 1
            else:
                # It was a tie
                new_df['Ties'][away_team_index] += 1
                new_df['Ties'][home_team_index] += 1
    
    return_df = pd.DataFrame.from_dict(new_df)
    
    return return_df

# Clean Defensive Stats

In [ ]:
# Drop all offensive players that may have a defensive stat such as a tackle chasing down
# an interception or a fumble return
'''
No longer used as the excel doc has been saved past this point
defensive_stats = defensive_stats.drop(defensive_stats[defensive_stats['Pos'] == 'OL'].index)
defensive_stats = defensive_stats.drop(defensive_stats[defensive_stats['Pos'] == 'RG'].index)
defensive_stats = defensive_stats.drop(defensive_stats[defensive_stats['Pos'] == 'RT'].index)
defensive_stats = defensive_stats.drop(defensive_stats[defensive_stats['Pos'] == 'LG'].index)
defensive_stats = defensive_stats.drop(defensive_stats[defensive_stats['Pos'] == 'LT'].index)
defensive_stats = defensive_stats.drop(defensive_stats[defensive_stats['Pos'] == 'LG/RG'].index)
defensive_stats = defensive_stats.drop(defensive_stats[defensive_stats['Pos'] == 'C'].index)
defensive_stats = defensive_stats.drop(defensive_stats[defensive_stats['Pos'] == 'G'].index)
defensive_stats = defensive_stats.drop(defensive_stats[defensive_stats['Pos'] == 'T'].index)
defensive_stats = defensive_stats.drop(defensive_stats[defensive_stats['Pos'] == 'C'].index)
defensive_stats = defensive_stats.drop(defensive_stats[defensive_stats['Pos'] == '0'].index)
defensive_stats = defensive_stats.drop(defensive_stats[defensive_stats['Pos'] == 'QB'].index)
defensive_stats = defensive_stats.drop(defensive_stats[defensive_stats['Pos'] == 'RB'].index)
defensive_stats = defensive_stats.drop(defensive_stats[defensive_stats['Pos'] == 'WR'].index)
defensive_stats = defensive_stats.drop(defensive_stats[defensive_stats['Pos'] == 'K'].index)
defensive_stats = defensive_stats.drop(defensive_stats[defensive_stats['Pos'] == 'LS'].index)
defensive_stats = defensive_stats.drop(defensive_stats[defensive_stats['Pos'] == 'P'].index)
defensive_stats = defensive_stats.drop(defensive_stats[defensive_stats['Pos'] == 'TE'].index)
'''

# defensive_stats.to_csv('16to22DStats.csv')

# Data sets used originally that were replaced by more accurate data

In [ ]:
# Defensive Stats From 2016-2022
# These are no longer in use and just read the CSV that was created from it
d2016 = pd.read_excel('2016_player_stats.xlsx')
d2017 = pd.read_excel('2017_player_stats.xlsx')
d2018 = pd.read_excel('2018_player_stats.xlsx')
d2019 = pd.read_excel('2019_player_stats.xlsx')
d2020 = pd.read_excel('2020_player_stats.xlsx')
d2021 = pd.read_excel('2021_player_stats.xlsx')
d2022 = pd.read_excel('2022_player_stats.xlsx')

defensive_stats = pd.concat([d2016, d2017, d2018, d2019, d2020, d2021, d2022])
defensive_stats = defensive_stats.fillna(0)
defensive_stats.to_csv('16to22DStats.csv')